In [ ]:
#scrivere a che cosa servono
import numpy as np
import h5py
import squeezing_library as sqz
from scipy.optimize import minimize #used to minimize the collective variance
import time
import math
import matplotlib as plt
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, TwoSlopeNorm, LinearSegmentedColormap #creates instance normalization, discrete colormap
import pandas as pd
import matplotlib.cm as cm
from matplotlib.colors import Normalize

In [ ]:
#computes the collective variance

def calculate_varc(theta, R_on,R_off):
    #
    comp_s_off = R_off[0]+ 1j*R_off[1]
    comp_s_on = R_on[0]+ 1j*R_on[1]
    comp_i_on = R_on[2]+ 1j*R_on[3]
    comp_i_off = R_off[2]+ 1j*R_off[3]

    rot = np.exp(1j * theta)  # e^(i*theta)

    #rotation of angle theta
    comp_s_off_rot = comp_s_off*rot
    comp_s_on_rot = comp_s_on*rot
    comp_i_off_rot = comp_i_off*rot
    comp_i_on_rot = comp_i_on*rot

    I_s_off = np.array(comp_s_off_rot.real) #new IQ components after rotation
    Q_s_off = np.array(comp_s_off_rot.imag)

    I_s_on = np.array(comp_s_on_rot.real)
    Q_s_on = np.array(comp_s_on_rot.imag)

    I_i_on = np.array(comp_i_on_rot.real)
    Q_i_on = np.array(comp_i_on_rot.imag)

    I_i_off = np.array(comp_i_off_rot.real)
    Q_i_off = np.array(comp_i_off_rot.imag)

    R_on = np.stack([I_s_on, Q_s_on, I_i_on, Q_i_on])
    R_off = np.stack([I_s_off, Q_s_off, I_i_off, Q_i_off])

    #computation of covariance matrix
    sigma_meas_on = np.cov(R_on)
    sigma_meas_off = np.cov(R_off)
    identita = 0.25*np.eye(4)
    sigma = sigma_meas_on-sigma_meas_off+identita

    #collective variance
    var1 = sigma[0,0] + sigma[2,2] + 2*sigma[2,0]

    #sqz_x = 10*np.log((var1)/0.5)
    
    return var1

In [ ]:
#computes the collective variance

def calculate_var_1rot(theta, R_on,R_off):
    #
    comp_s_off = R_off[0]+ 1j*R_off[1]
    comp_s_on = R_on[0]+ 1j*R_on[1]
    comp_i_on = R_on[2]+ 1j*R_on[3]
    comp_i_off = R_off[2]+ 1j*R_off[3]

    rot = np.exp(1j * theta)  # e^(i*theta)

    #rotation of angle theta
    comp_s_off_rot = comp_s_off
    comp_s_on_rot = comp_s_on
    comp_i_off_rot = comp_i_off*rot
    comp_i_on_rot = comp_i_on*rot

    I_s_off = np.array(comp_s_off_rot.real) #new IQ components after rotation
    Q_s_off = np.array(comp_s_off_rot.imag)

    I_s_on = np.array(comp_s_on_rot.real)
    Q_s_on = np.array(comp_s_on_rot.imag)

    I_i_on = np.array(comp_i_on_rot.real)
    Q_i_on = np.array(comp_i_on_rot.imag)

    I_i_off = np.array(comp_i_off_rot.real)
    Q_i_off = np.array(comp_i_off_rot.imag)

    R_on = np.stack([I_s_on, Q_s_on, I_i_on, Q_i_on])
    R_off = np.stack([I_s_off, Q_s_off, I_i_off, Q_i_off])

    #computation of covariance matrix
    sigma_meas_on = np.cov(R_on)
    sigma_meas_off = np.cov(R_off)
    identita = 0.25*np.eye(4)
    sigma = sigma_meas_on-sigma_meas_off+identita

    #collective variance
    var1 = sigma[0,0] + sigma[2,2] + 2*sigma[2,0]

    #sqz_x = 10*np.log((var1)/0.5)
    
    return var1

In [ ]:
#2d histograms
def new_hist(R_on, R_off, nbins: int, bin_edges: np.ndarray, norm, x_mode1: bool, p_mode1: bool, x_mode2: bool, p_mode2: bool, tick):
    # Map of modes (x and p) for each case
    modes = [(x_mode1, '$x_{s}$', '$\\bar{x}_{s}$', R_on[0], R_off[0]), 
             (p_mode1, '$p_{s}$', '$\\bar{p}_{s}$', R_on[1], R_off[1]),
             (x_mode2, '$x_{i}$', '$\\bar{x}_{i}$', R_on[2], R_off[2]),
             (p_mode2, '$p_{i}$', '$\\bar{p}_{i}$', R_on[3], R_off[3])]
    
    # Initialize the variables
    #alpha = [x_mode1, p_mode1, x_mode2, p_mode2]
    c = 0
    x, y = [], []
    #x_mean_on, x_mean_off, y_mean_on, y_mean_off = 0, 0, 0, 0

    # For loop to check which modes to use for 2d histogram
    for mode, name, mean_name, R_on_mode, R_off_mode in modes:
        if mode:
            if c == 0:
                x_name = name
                #x_mean_name = mean_name
                #x_mean_on, x_mean_off = np.mean(R_on_mode), np.mean(R_off_mode)
                x = np.concatenate([R_on_mode, R_off_mode])
                c = 1
            else:
                y_name = name
                #y_mean_name = mean_name
                #y_mean_on, y_mean_off = np.mean(R_on_mode), np.mean(R_off_mode)
                y = np.concatenate([R_on_mode, R_off_mode])
    
    # Create the histogram
    g = plt.hist2d(
        x, 
        y, 
        weights=np.concatenate([np.ones(len(R_on[0]))/(len(R_on[0]))*100, -np.ones(len(R_on[0]))/(len(R_on[0]))*100]),
        bins=(nbins, nbins), 
        range=(bin_edges, bin_edges), 
        cmap='bwr', 
        norm=norm
    )

    # Axes settings
    plt.xlabel(x_name, fontsize=15)
    plt.ylabel(y_name, fontsize=15)
    plt.tick_params(axis='both', labelsize=12)
    plt.xticks(np.arange(bin_edges[0], bin_edges[1] + 1, tick))
    plt.yticks(np.arange(bin_edges[0], bin_edges[1] + 1, tick))

    return g

#cosa fa di diverso da prima?
#Abbiamo mappato le variabili (x_mode1, p_mode1, etc.) in una lista chiamata modes, 
#che contiene tuple con i dati e le etichette associate. Questo consente di evitare la duplicazione di codice 
#e facilita la gestione delle modalità.
#Utilizziamo np.concatenate per unire gli array R_on_mode e R_off_mode invece di usare 
#[*R_on[i], *R_off[i]], che è più efficiente e più chiaro.

In [ ]:
#Data acquisition
def get_IQ_from_file(file: h5py._hl.files.File):
    try:
        signal_pump_off = file["Channel_acquire_signal_pump_off_0_Acquisition_0"]
        I_s_off = np.array(signal_pump_off["iq_real"])
        Q_s_off = np.array(signal_pump_off["iq_imaginary"])

        signal_pump_on = file["Channel_acquire_signal_pump_on_0_Acquisition_0"]
        I_s_on = np.array(signal_pump_on["iq_real"])
        Q_s_on = np.array(signal_pump_on["iq_imaginary"])

        idler_pump_on = file["Channel_acquire_idler_pump_on_0_Acquisition_0"]
        I_i_on = np.array(idler_pump_on["iq_real"])
        Q_i_on = np.array(idler_pump_on["iq_imaginary"])

        idler_pump_off = file["Channel_acquire_idler_pump_off_0_Acquisition_0"]
        I_i_off = np.array(idler_pump_off["iq_real"])
        Q_i_off = np.array(idler_pump_off["iq_imaginary"])
    except KeyError:
        print("Variable not found")
        return -1
    
    R_on = np.stack([I_s_on, Q_s_on, I_i_on, Q_i_on])
    R_off = np.stack([I_s_off, Q_s_off, I_i_off, Q_i_off])

    return R_on, R_off

In [ ]:
def get_IQ_after_rot(R_on: np.ndarray,
                     R_off: np.ndarray,
                     delta: float #rotation angle in rad
                     ):
    comp = np.empty(len(R_on), dtype=object) #needed to perform a sum of R_on and R_off components
    comp[0] = R_on[0] + 1j*R_on[1]
    comp[1] = R_on[2] + 1j*R_on[3]
    comp[2] = R_off[0] + 1j*R_off[1]
    comp[3] = R_off[2] + 1j*R_off[3]

    #rotation of angle delta (radians)
    delta_rot = np.exp(1j*delta)
    comp = comp*delta_rot

    I_s_on = np.array(comp[0].real)
    Q_s_on = np.array(comp[0].imag)

    I_i_on = np.array(comp[1].real)
    Q_i_on = np.array(comp[1].imag)
    
    I_s_off = np.array(comp[2].real)
    Q_s_off = np.array(comp[2].imag)

    I_i_off = np.array(comp[3].real)
    Q_i_off = np.array(comp[3].imag)

    R_on = np.stack([I_s_on, Q_s_on, I_i_on, Q_i_on])
    R_off = np.stack([I_s_off, Q_s_off, I_i_off, Q_i_off])

    return R_on, R_off

In [ ]:
 # for each quadrature, using iq function, we reconstruct sqz.iq quadrature at dut output level
def iq_at_dut(R_on: np.ndarray,
              R_off: np.ndarray,
              volt: bool, #true if the output needs to be in volts
              f,
              G,
              tau,
              Z): #parameters for the sqz.iq function, list (or arrays) of 2 elements
    R_on_dut = []
    R_off_dut = []
    if volt:
        for p in range(4):
            if p < 2:
                R_on_dut.append(sqz.iq(R_on[p]*1.6, f[0], G[0], tau, Z))
                R_off_dut.append(sqz.iq(R_off[p]*1.6, f[0], G[0], tau, Z))
            else:
                R_on_dut.append(sqz.iq(R_on[p]*1.6, f[1], G[1], tau, Z))
                R_off_dut.append(sqz.iq(R_off[p]*1.6, f[1], G[1], tau, Z))
    else:
        for p in range(4):
            if p < 2:
                R_on_dut.append(sqz.iq(R_on[p], f[0], G[0], tau, Z))
                R_off_dut.append(sqz.iq(R_off[p], f[0], G[0], tau, Z))
            else:
                R_on_dut.append(sqz.iq(R_on[p], f[1], G[1], tau, Z))
                R_off_dut.append(sqz.iq(R_off[p], f[1], G[1], tau, Z))

    return R_on_dut, R_off_dut

This is the data analysis of a two mode squeezing experiment in a Traveling Wave Parametric Amplifier.

Import data
-----------------

In [ ]:
#import data
s = time.perf_counter()
file_demod = h5py.File(r'Data/argo_2024-11-08_15-29.hdf5')
#file_demod = h5py.File(r"Data/argo_2024-09-20_16-07.hdf5")

In [ ]:
print(list(file_demod.attrs.keys()))
print(file_demod.attrs["FPGAPostprocessing"])
chan = file_demod.attrs["Program"]

In [ ]:
#create the list with frequency informations
frequencies = sqz.extract_info(chan, 'Frequency')

#extract info prende due stringhe come argomento. Restituisce valori delle frequenze
for idx, result in enumerate(frequencies):
    print(f"{idx}: {result}")
#print(frequencies)

In [ ]:
#extract the values
frequencies_list = sqz.extract_values_from_list(frequencies)
print(frequencies_list)

In [ ]:
R_on, R_off = get_IQ_from_file(file_demod)
print(R_on[0])

In [ ]:
#import gain file
file_csv = r'gains.csv'

In [ ]:
#experimental setup
tau = 6e-6 #6 microsecondi, tempo di acquisizione
Z = 50 #impedenza, 50 ohm
fs = frequencies_list[1] #frequenze del segnale
fi = frequencies_list[2] #frequenze dell'idler
G_s = sqz.db_to_linear(sqz.gain_finder(file_csv, fs)[1]+10)  #74 dB 75
G_i = sqz.db_to_linear(sqz.gain_finder(file_csv, fi)[1]+10) #74 dB 75
print(f"G_s = {G_s}\nG_i = {G_i}")

In [ ]:
dB = sqz.gain_finder(file_csv, fs)[1]+10
print(dB)

In [ ]:
f = [fs, fi]
G = [G_s, G_i]
R_on_dut, R_off_dut = iq_at_dut(R_on, R_off, False, f, G, tau, Z)

In [ ]:
print(R_on_dut[0])

In [ ]:
#minimize the collective variance
#x0 should not be near the interval endpoints
res = minimize(lambda theta: calculate_varc(theta,R_on_dut,R_off_dut), x0=[1], bounds=[(0, 2*np.pi)])
res.x[0]

In [ ]:
#check if the rotation of only one system of axes takes less time -> Answer: it takes the same time
#minimize the collective variance
#x0 should not be near the interval endpoints
res2 = minimize(lambda theta: calculate_var_1rot(theta,R_on_dut,R_off_dut), x0=[1], bounds=[(0, 2*np.pi)])
res2.x[0]

In [ ]:
#use the optimal angle to find the new components
delta_theta = res.x[0]
gradi = math.degrees(delta_theta)
print(f"Angolo in gradi: {gradi}")

#computation of x_s, p_s, x_i, p_i (on/off)
R_on, R_off = get_IQ_after_rot(R_on, R_off, delta_theta)
print(R_on[0])

In [ ]:
#reconstruct quadratures at dut output level (in volts)
R_on, R_off = iq_at_dut(R_on, R_off, True, f, G, tau, Z)
print(R_on[0])

In [ ]:
bin_edges = (-5, 5)  
nbins=500
vmin = -50  # Limite inferiore
vmax = 50  # Limite superiore
norm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)  # Crea l'istanza di normalizzazione con vmin e vmax
tick = 5

fig = plt.figure(figsize=(8, 7.5)) # Crea il colorplot dell'istogramma 2D

# x_mode1: bool, p_mode1: bool, x_mode2: bool, p_mode2: bool
plt.subplot(2, 2, 1)  # 1 row, 2 columns, 2nd subplot
plt.hist2d(R_on[0], R_on[1], bins=(nbins, nbins), range=(bin_edges,bin_edges), cmap='bwr', norm=norm,)
plt.xlabel("$x_{son}$", fontsize=15)
plt.ylabel("$p_{son}$", fontsize=15)
plt.subplot(2, 2, 2)  # 1 row, 2 columns, 2nd subplot
counts, xedges, yedges, im = plt.hist2d(R_on[2], R_on[3], bins=(nbins, nbins), range=(bin_edges,bin_edges), cmap='bwr', norm=norm,)
plt.xlabel("$x_{ion}$", fontsize=15)
plt.ylabel("$p_{ion}$", fontsize=15)
plt.subplot(2, 2, 3)  # 1 row, 2 columns, 2nd subplot
counts, xedges, yedges, im = plt.hist2d(R_off[0], R_off[1], bins=(nbins, nbins), range=(bin_edges,bin_edges), cmap='bwr', norm=norm,)
plt.xlabel("$x_{soff}$", fontsize=15)
plt.ylabel("$p_{soff}$", fontsize=15)
plt.subplot(2, 2, 4)  # 1 row, 2 columns, 1st subplot
counts, xedges, yedges, im = plt.hist2d(R_off[2], R_off[3], bins=(nbins, nbins), range=(bin_edges,bin_edges), cmap='bwr', norm=norm,)
plt.xlabel("$x_{ioff}$", fontsize=15)
plt.ylabel("$p_{ioff}$", fontsize=15)
# Aggiunta di una colorbar personalizzata
cbar_ax = fig.add_axes([1.05, 0.1, 0.02, 0.8])  # [x0, y0, width, height]
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Counts', fontsize=15)
cbar.ax.tick_params(labelsize=12)
plt.tight_layout()
plt.show()

Pump on - pump off istograms
-----------------

In [ ]:
#2d histograms
bin_edges = (-5, 5)  
nbins=200
vmin = -0.01  # Limite inferiore
vmax = 0.01  # Limite superiore
norm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)  # Crea l'istanza di normalizzazione con vmin e vmax
tick = 5

fig = plt.figure(figsize=(12, 7.5)) # Crea il colorplot dell'istogramma 2D

# x_mode1: bool, p_mode1: bool, x_mode2: bool, p_mode2: bool
plt.subplot(2, 3, 1)  # 1 row, 2 columns, 2nd subplot
new_hist(R_on, R_off, nbins, bin_edges, norm, True, True, False, False,tick)
plt.subplot(2, 3, 2)  # 1 row, 2 columns, 2nd subplot
new_hist(R_on, R_off, nbins, bin_edges, norm, False, False, True, True,tick)
plt.subplot(2, 3, 3)  # 1 row, 2 columns, 2nd subplot
new_hist(R_on, R_off, nbins, bin_edges, norm, True, False, True, False,tick)
plt.subplot(2, 3, 4)  # 1 row, 2 columns, 1st subplot
new_hist(R_on, R_off, nbins, bin_edges, norm, False, True, True, False,tick)
plt.subplot(2, 3, 5)  # 1 row, 2 columns, 2nd subplot
new_hist(R_on, R_off, nbins, bin_edges, norm, True, False, False, True,tick)
plt.subplot(2, 3, 6)  # 1 row, 2 columns, 2nd subplot
g = new_hist(R_on, R_off, nbins, bin_edges, norm, False, True, False, True,tick)
# Aggiunta di una colorbar personalizzata
cbar_ax = fig.add_axes([1.05, 0.1, 0.02, 0.8])  # [x0, y0, width, height]
cbar = fig.colorbar(g[3], cax=cbar_ax)
cbar.set_label('Counts on - counts off [%]', fontsize=15)
cbar.ax.tick_params(labelsize=12)
plt.tight_layout()
plt.show()
#da 8-9 secondi siamo passati a 2 dopo avere snellito il codice!!

In [ ]:
on = np.sqrt(R_on[0]**2 + R_on[1]**2)
# Calcola la deviazione standard delle distanze
std_deviation_on = np.std(on)

off = np.sqrt(R_off[0]**2 + R_off[1]**2)
# Calcola la deviazione standard delle distanze
std_deviation_off = np.std(off)

gain = std_deviation_on/std_deviation_off
gain

In [ ]:
on = np.sqrt(R_on[2]**2 + R_on[3]**2)
# Calcola la deviazione standard delle distanze
std_deviation_on = np.std(on)

off = np.sqrt(R_off[2]**2 + R_off[3]**2)
# Calcola la deviazione standard delle distanze
std_deviation_off = np.std(off)

gain = std_deviation_on/std_deviation_off
gain

Thermal States Gaussianity
---------------------

In [ ]:
# x_mode1: bool, p_mode1: bool, x_mode2: bool, p_mode2: bool
par = sqz.gaussianity_proj(R_off, True, False, True, False)
sigma = par[2]
print(sigma)

$\sigma^{meas,on} = \sigma _{|TMS>} + \sigma _{thermal}$


$\sigma^{meas,off} = \sigma _{|0>} + \sigma _{thermal}$


$\sigma _{|TMS>} = \sigma^{meas,on} - \sigma^{meas,off} + \sigma _{|0>}$

Reference: https://arxiv.org/pdf/2111.03696

Covariance matrix
-----------------

In [ ]:
#covariance matrix
sigma_meas_on = np.cov(R_on)
sigma_meas_off = np.cov(R_off)
identita = 0.25*np.eye(4)
sigma = sigma_meas_on-sigma_meas_off+identita
sigma

In [ ]:
#covariance matrix plot

labels = ["$x_{s}$", "$p_{s}$", "$x_{i}$", "$p_{i}$"]  # Etichette degli assi
x, y = np.meshgrid(np.arange(sigma.shape[0]), np.arange(sigma.shape[1]))  # Creazione della griglia per l'istogramma 2D
x = x.ravel()
y = y.ravel()
z = np.zeros_like(x)
dz = sigma.ravel()   # Altezze delle barre (elementi della matrice di covarianza)
dx = dy = 0.8  # Dimensioni delle barre
colors = np.where(dz >= 0, 'red', 'blue')  # Creazione di una mappa di colori personalizzata per positivo/negativo
alpha_values = np.where(dz >= 0, 0.8, 0.8)
fig_cov_matrix = plt.figure(figsize=(8, 8))  # Creazione del plot 3D
fig_cov_matrix.patch.set_alpha(0)  # Sfondo trasparente della figura
ax = fig_cov_matrix.add_subplot(111, projection='3d')

#bar_plot = ax.bar3d(x, y, z, dx, dy, dz, shade=True, color=colors, alpha = 1 , zsort='average')  # Grafico a barre 3D

for xi, yi, zi, dzi, color, alpha in zip(x, y, z, dz, colors, alpha_values):
    ax.bar3d(xi, yi, zi, dx, dy, dzi, shade=True, color=color, alpha=alpha, edgecolor='k',linewidth=1 ,zsort='average')



ax.view_init(elev=30, azim=-140)  # Impostazione dell'angolo di vista
#ax.set_zlabel('Covariance')  # Etichette degli assi
ax.set_xticks(np.arange(len(labels)))
ax.set_yticks(np.arange(len(labels)))
ax.set_xticklabels(labels)
ax.set_yticklabels(labels)
cmap = LinearSegmentedColormap.from_list('red_blue', ['blue', 'red'], N=2)  # Creazione di una mappa di colori discreta (rosso/blu)
norm = Normalize(vmin=-1, vmax=1)
sm = cm.ScalarMappable(cmap=cmap, norm=norm)   # Aggiunta della barra colori
sm.set_array([])  # Necessario per evitare avvisi
#plt.title("Covariance Matrix")  # Mostra il grafico
plt.show()

Squeezing
----------

Paragraph 4.5.2 Eichler PhD thesis

In [ ]:
#Squeezing
# X- = xs - xi
# Var(X−Y)=Var(X)+Var(Y)−2⋅Cov(X,Y)
var1 = sigma[0,0] + sigma[2,2] - abs(2*sigma[2,0])
var1

# P+ = ps + pi
# Var(X+Y)=Var(X)+Var(Y)+2⋅Cov(X,Y)
var2 = sigma[1,1] + sigma[3,3] - abs(2*sigma[1,3])
var2

sqz_x = 10*np.log((var1)/0.5)
sqz_p = 10*np.log((var2)/0.5)

print(sqz_x)
print(sqz_p)
e = time.perf_counter()
print(f"Time elapsed: {e-s}")

Logarithmic negativity
---------------

In [ ]:
print(sqz.log_neg(sigma))

Wigner function reconstruction
----------------------

For Gaussian states,

$W(\alpha) = \frac{1}{4\pi^2 \sqrt{\text{det}(V)}} \exp \left( -\frac{1}{2} \alpha V^{-1} \alpha^T \right)$

Reference: https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.107.113601


In [ ]:
X_grid = Y_grid = np.linspace(-3, 3, 100)  # (X,Y) grid on which we compute Wigner function
colors = [(0., 'darkblue'), (0.2, 'blue'), (0.4, 'skyblue'), (0.6, 'yellow'), (0.8, 'red'), (1., 'brown')] 
cmap = LinearSegmentedColormap.from_list('custom_cmap', colors)
#cmap = 'viridis'
vmin = 0.0    # colormap limits
vmax= 0.23
v = np.linspace(vmin,vmax,6)
vcenter = vmax/2
norm = TwoSlopeNorm(vmin=vmin,vcenter = vcenter, vmax=vmax)  # Crea l'istanza di normalizzazione con vmin e vmax
up_ins = 2   # iset limits
low_ins = -2

fig = plt.figure(figsize=(12, 8))

plt.subplot(2, 3, 1)    # ideal vacuum state
xs,ps,xi,pi = True, True, False, False   # whether to project onto this dimension or not
W_values_vac = sqz.wigner_proj(identita, X_grid, Y_grid, xs, ps, xi, pi)   # wigner function projection on the desired dimesnions on the grid
sqz.wigner_plot('Ideal vacuum state', W_values_vac, cmap, X_grid, Y_grid, vmin, vmax, xs,ps,xi,pi)   # wigner function colorplot
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)

plt.subplot(2, 3, 4)   # ideal thermal state
T = 0.15  # temperature of thermal state
V_the = (2*sqz.N(fs,T)+1)*identita
xs,ps,xi,pi = True, True, False, False
W_values_the = sqz.wigner_proj(V_the ,X_grid, Y_grid, xs,ps,xi,pi)
g = sqz.wigner_plot('Ideal thermal state', W_values_the, cmap, X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
#sqz.inset_constructor_2(X_grid, Y_grid,V_the,xs,ps,xi,pi, low_ins, up_ins, 'green',3)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)

plt.subplot(2, 3, 2)  # Reconstructed state xs ps
xs,ps,xi,pi = True, True, False, False
V_the = (2*sqz.N(fs,T)+1)*identita
W_values = sqz.wigner_proj(sigma,X_grid, Y_grid, xs,ps,xi,pi)
sqz.wigner_plot('Reconstructed state', W_values, cmap, X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)
sqz.level_curve(ax_inset,sigma, xs,ps,xi,pi,"red",3)

plt.subplot(2, 3, 5)  # Reconstructed state xi pi
xs,ps,xi,pi = False, False, True, True
V_the = (2*sqz.N(fi,T)+1)*identita
W_values = sqz.wigner_proj(sigma, X_grid, Y_grid,xs,ps,xi,pi)
sqz.wigner_plot('Reconstructed state', W_values, cmap, X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)
sqz.level_curve(ax_inset,sigma, xs,ps,xi,pi,"red",3)

plt.subplot(2, 3, 3)  # Reconstructed state xs xi
xs,ps,xi,pi = True, False, True, False
V_the = (2*sqz.N(fs,T)+1)*identita
W_values = sqz.wigner_proj(sigma, X_grid, Y_grid,xs,ps,xi,pi)
sqz.wigner_plot('Reconstructed state', W_values, cmap,X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)
sqz.level_curve(ax_inset,sigma, xs,ps,xi,pi,"red",3)

plt.subplot(2, 3, 6)  # Reconstructed state ps pi
xs,ps,xi,pi = False, True, False, True
W_values = sqz.wigner_proj(sigma, X_grid, Y_grid,xs,ps,xi,pi)
sqz.wigner_plot('Reconstructed state', W_values, cmap, X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)
sqz.level_curve(ax_inset,sigma, xs,ps,xi,pi,"red",3)

# Aggiunta di una colorbar personalizzata
cbar_ax = fig.add_axes([1.05, 0.1, 0.02, 0.8])  # [x0, y0, width, height]
# Aggiungere la colorbar
cbar = fig.colorbar(g, cax=cbar_ax)
cbar.ax.tick_params(labelsize=12)
plt.tight_layout()
plt.show()

Thermal noise
--------------

In [ ]:
X_grid = Y_grid = np.linspace(-5, 5, 100)  # (X,Y) grid on which we compute Wigner function
colors = [(0., 'darkblue'), (0.2, 'blue'), (0.4, 'skyblue'), (0.6, 'yellow'), (0.8, 'red'), (1., 'brown')] 
cmap = LinearSegmentedColormap.from_list('custom_cmap', colors)
#cmap = 'viridis'
vmin = 0.0    # colormap limits
vmax= 0.001
v = np.linspace(vmin,vmax,6)
vcenter = vmax/2
norm = TwoSlopeNorm(vmin=vmin,vcenter = vcenter, vmax=vmax)  # Crea l'istanza di normalizzazione con vmin e vmax
up_ins = 5   # iset limits
low_ins = -5

fig = plt.figure(figsize=(12, 8))
plt.subplot(2, 3, 1)    # ideal vacuum state
xs,ps,xi,pi = True, True, False, False   # whether to project onto this dimension or not
W_values_vac = sqz.wigner_proj((sigma_meas_off-identita), X_grid, Y_grid, xs, ps, xi, pi)   # wigner function projection on the desired dimensions on the grid
g = sqz.wigner_plot('', W_values_vac, cmap, X_grid, Y_grid, vmin, vmax, xs,ps,xi,pi)   # wigner function colorplot
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,(sigma_meas_off-identita), xs,ps,xi,pi,"red",3)
T=3
V_the = (2*sqz.N(fs,T)+1)*identita
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)

In [ ]:
X_grid = Y_grid = np.linspace(-3, 3, 100)  # (X,Y) grid on which we compute Wigner function
colors = [(0., 'darkblue'), (0.2, 'blue'), (0.4, 'skyblue'), (0.6, 'yellow'), (0.8, 'red'), (1., 'brown')] 
cmap = LinearSegmentedColormap.from_list('custom_cmap', colors)
#cmap = 'viridis'
vmin = 0.0    # colormap limits
vmax= 0.01
v = np.linspace(vmin,vmax,6)
vcenter = vmax/2
norm = TwoSlopeNorm(vmin=vmin,vcenter = vcenter, vmax=vmax)  # Crea l'istanza di normalizzazione con vmin e vmax
up_ins = 2   # iset limits
low_ins = -2

fig = plt.figure(figsize=(8, 8))


plt.subplot(2, 2, 1)  # Reconstructed state xs ps
T = 0.15
xs,ps,xi,pi = True, True, False, False
V_the = (2*sqz.N(fs,T)+1)*identita
W_values = sqz.wigner_proj(sigma,X_grid, Y_grid, xs,ps,xi,pi)
g = sqz.wigner_plot('Reconstructed state', W_values, cmap, X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)
sqz.level_curve(ax_inset,sigma, xs,ps,xi,pi,"red",3)

plt.subplot(2, 2, 3)  # Reconstructed state xi pi
xs,ps,xi,pi = False, False, True, True
V_the = (2*sqz.N(fi,T)+1)*identita
W_values = sqz.wigner_proj(sigma, X_grid, Y_grid,xs,ps,xi,pi)
sqz.wigner_plot('Reconstructed state', W_values, cmap, X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)
sqz.level_curve(ax_inset,sigma, xs,ps,xi,pi,"red",3)

plt.subplot(2, 2, 2)  # Reconstructed state xs xi
xs,ps,xi,pi = True, False, True, False
V_the = (2*sqz.N(fs,T)+1)*identita
W_values = sqz.wigner_proj(sigma, X_grid, Y_grid,xs,ps,xi,pi)
sqz.wigner_plot('Reconstructed state', W_values, cmap,X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)
sqz.level_curve(ax_inset,sigma, xs,ps,xi,pi,"red",3)

plt.subplot(2, 2, 4)  # Reconstructed state ps pi
xs,ps,xi,pi = False, True, False, True
W_values = sqz.wigner_proj(sigma, X_grid, Y_grid,xs,ps,xi,pi)
sqz.wigner_plot('Reconstructed state', W_values, cmap, X_grid, Y_grid, vmin, vmax,xs,ps,xi,pi)
ax_inset = sqz.inset_constructor_2(low_ins, up_ins)
sqz.level_curve(ax_inset,identita, xs,ps,xi,pi,"blue",3)
sqz.level_curve(ax_inset,V_the, xs,ps,xi,pi,"green",3)
sqz.level_curve(ax_inset,sigma, xs,ps,xi,pi,"red",3)

# Aggiunta di una colorbar personalizzata
cbar_ax = fig.add_axes([1.05, 0.1, 0.02, 0.8])  # [x0, y0, width, height]
# Aggiungere la colorbar
cbar = fig.colorbar(g, cax=cbar_ax)
cbar.ax.tick_params(labelsize=12)
plt.tight_layout()
plt.show()

In [ ]:
#Proprietà della matrice di covarianza
# Calcolare gli autovalori
eigenvalues = np.linalg.eigvals(sigma)

# Calcolare il numero di condizionamento (in termini di norma)
cond_number = np.linalg.cond(sigma)

# Stampa degli autovalori e del numero di condizionamento
print("Autovalori della matrice di covarianza:", eigenvalues)
print("Numero di condizionamento della matrice:", cond_number)

# Verifica se la matrice è ben condizionata
if np.all(eigenvalues > 0) and cond_number < 1e10:
    print("La matrice è ben condizionata.")
else:
    print("La matrice è mal condizionata.")
    #eventualmente implementare un algoritmo per una matrice di covarianza più robusta